In [1]:
import cv2
import numpy as np
import csv
import datetime
import os
from collections import OrderedDict

In [ ]:


class CentroidTracker:
    def __init__(self, max_disappeared=50, csv_writer=None):
        self.next_object_id = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        
        # Timing storage
        self.start_times = OrderedDict()
        self.max_disappeared = max_disappeared
        self.csv_writer = csv_writer

    def register(self, centroid):
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        
        # Store absolute start time
        self.start_times[self.next_object_id] = datetime.datetime.now()
        self.next_object_id += 1

    def deregister(self, object_id):
        # Calculate duration
        end_time = datetime.datetime.now()
        start_time = self.start_times[object_id]
        duration = (end_time - start_time).total_seconds()
        
        # Save to CSV immediately when person leaves
        if self.csv_writer:
            self.csv_writer.writerow([
                object_id, 
                start_time.strftime("%Y-%m-%d %H:%M:%S"), 
                end_time.strftime("%Y-%m-%d %H:%M:%S"), 
                round(duration, 2),
                "Left Video"
            ])
            print(f"Logged ID {object_id}: {duration:.2f}s")

        del self.objects[object_id]
        del self.disappeared[object_id]
        del self.start_times[object_id]

    def update(self, rects):
        if len(rects) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        input_centroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            input_centroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(input_centroids)):
                self.register(input_centroids[i])
        else:
            object_ids = list(self.objects.keys())
            object_centroids = list(self.objects.values())

            D = []
            for oc in object_centroids:
                row = []
                for ic in input_centroids:
                    dist = np.linalg.norm(np.array(oc) - np.array(ic))
                    row.append(dist)
                D.append(row)
            D = np.array(D)

            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            used_rows = set()
            used_cols = set()

            for (row, col) in zip(rows, cols):
                if row in used_rows or col in used_cols:
                    continue

                object_id = object_ids[row]
                self.objects[object_id] = input_centroids[col]
                self.disappeared[object_id] = 0
                used_rows.add(row)
                used_cols.add(col)

            unused_rows = set(range(0, D.shape[0])).difference(used_rows)
            unused_cols = set(range(0, D.shape[1])).difference(used_cols)

            for row in unused_rows:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

            for col in unused_cols:
                self.register(input_centroids[col])

        return self.objects

def main():
    # 1. Setup CSV file
    csv_file = open('tracking_data.csv', 'w', newline='')
    writer = csv.writer(csv_file)
    writer.writerow(["Person ID", "Start Time", "End Time", "Duration (Sec)", "Status"])

    # 2. Initialize Tracker with writer
    ct = CentroidTracker(max_disappeared=30, csv_writer=writer)
    
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    video_path = r"E:\AI\datasets\befoys-webcams\befoys local_ch6_20251208155651_20251208162738.mp4"
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video")
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, (640, 480))
            (rects, _) = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05)

            box_rects = []
            for (x, y, w, h) in rects:
                box_rects.append((x, y, x + w, y + h))
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            objects = ct.update(box_rects)

            for (object_id, centroid) in objects.items():
                # Real-time duration display
                duration = (datetime.datetime.now() - ct.start_times[object_id]).total_seconds()
                text = f"ID {object_id}: {duration:.1f}s"
                cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

            cv2.imshow("Frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # 3. Save remaining objects when script ends
        print("Saving remaining objects...")
        end_time = datetime.datetime.now()
        for object_id in list(ct.objects.keys()):
            start_time = ct.start_times[object_id]
            duration = (end_time - start_time).total_seconds()
            writer.writerow([
                object_id, 
                start_time.strftime("%Y-%m-%d %H:%M:%S"), 
                end_time.strftime("%Y-%m-%d %H:%M:%S"), 
                round(duration, 2),
                "End of Video"
            ])
        
        cap.release()
        cv2.destroyAllWindows()
        csv_file.close()
        print("Data saved to tracking_data.csv")

if __name__ == "__main__":
    main()


Logged ID 0: 7.31s
Logged ID 1: 30.12s
Saving remaining objects...
Data saved to tracking_data.csv
